In [9]:
import pandas as pd
from datetime import datetime, timedelta

In [10]:
order_items=pd.read_csv('/Users/jiin/dacon/KPI_Dacon_2024/data/order_items.csv')
orders=pd.read_csv('/Users/jiin/dacon/KPI_Dacon_2024/data/orders.csv')

In [11]:
#주문상태가 delivered 인 주문만 
orders=orders[orders['Order_status']=='delivered']

In [12]:
orders.head()

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
0,ORDER_00000,CUSTOMER_62232,delivered,2018-10-02 10:56:33,2018-10-04 19:55:00,2018-10-10 21:25:13,2018-10-18
1,ORDER_00001,CUSTOMER_54059,delivered,2018-11-18 19:28:06,2018-11-22 13:39:59,2018-12-02 00:28:42,2018-12-15
2,ORDER_00002,CUSTOMER_59535,delivered,2019-02-13 21:18:39,2019-02-14 19:46:34,2019-02-16 18:17:02,2019-02-26
3,ORDER_00003,CUSTOMER_60516,delivered,2018-07-09 21:57:05,2018-07-11 14:58:04,2018-07-26 10:57:55,2018-08-01
4,ORDER_00004,CUSTOMER_76957,delivered,2018-05-16 13:10:30,2018-05-22 10:07:46,2018-05-26 12:55:51,2018-06-07


In [13]:
order_items.head()

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.9,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.9,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.0,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.9,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.9,12.69


In [14]:
orders['Order_purchase_timestamp']=pd.to_datetime(orders['Order_purchase_timestamp'])

In [15]:
seller_order=pd.merge(orders,order_items,on='Order_id')
seller_order.head()

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_00000,CUSTOMER_62232,delivered,2018-10-02 10:56:33,2018-10-04 19:55:00,2018-10-10 21:25:13,2018-10-18,1,PRODUCT_02093,SELLER_0491,29.99,8.72
1,ORDER_00001,CUSTOMER_54059,delivered,2018-11-18 19:28:06,2018-11-22 13:39:59,2018-12-02 00:28:42,2018-12-15,1,PRODUCT_26747,SELLER_2662,45.00,27.20
2,ORDER_00002,CUSTOMER_59535,delivered,2019-02-13 21:18:39,2019-02-14 19:46:34,2019-02-16 18:17:02,2019-02-26,1,PRODUCT_10598,SELLER_1354,19.90,8.72
3,ORDER_00003,CUSTOMER_60516,delivered,2018-07-09 21:57:05,2018-07-11 14:58:04,2018-07-26 10:57:55,2018-08-01,1,PRODUCT_16703,SELLER_1518,147.90,27.36
4,ORDER_00004,CUSTOMER_76957,delivered,2018-05-16 13:10:30,2018-05-22 10:07:46,2018-05-26 12:55:51,2018-06-07,1,PRODUCT_08274,SELLER_1599,59.99,15.17


## RFM 생성
- Recency (R): 가장 최근 구매로부터 경과한 시간
- Frequency (F): 총 구매 횟수
- Monetary (M): 총 구매 금액

여기서 궁금한 점. Frequency가 한번에 살때 20개 산것도 구매횟수로 치는건지 아니면 몇개를 사던 상관없이 한번 주문했을 때를 횟수는 치는 건지 궁금.


In [17]:
#주문별 총 구매 금액 계산
seller_order['Total']=seller_order['Price']+seller_order['Freight_value']


In [18]:
#Frequency
frequency=seller_order.groupby('Seller_id').size().reset_index(name='Frequency')

#Monetary
monetary=seller_order.groupby('Seller_id')['Total'].sum().reset_index()

#Recency
current_date=seller_order['Order_purchase_timestamp'].max()+timedelta(days=1)
seller_order['Recency']=(current_date-pd.to_datetime(seller_order['Order_purchase_timestamp'])).dt.days
recency=seller_order.groupby('Seller_id')['Recency'].min().reset_index()

rfm=pd.merge(recency,frequency,on='Seller_id')
rfm=pd.merge(rfm,monetary,on='Seller_id')

In [19]:
rfm

,Seller_id,Recency,Frequency,Total
0,SELLER_0000,335,3,246.60
1,SELLER_0001,54,40,12823.32
2,SELLER_0002,259,1,199.92
3,SELLER_0003,83,1,73.89
4,SELLER_0004,5,35,4765.66
...,...,...,...,...
2756,SELLER_2758,17,1,93.85
2757,SELLER_2759,210,7,851.80
2758,SELLER_2760,9,4,505.58
2759,SELLER_2761,483,10,442.99


## RFM score

In [20]:
#R,F,M 값에 대해 4개의 구간으로 나누어 점수 부여
rfm['R']=pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
rfm['F']=pd.qcut(rfm['Frequency'].rank(method='first'),5,labels=[1,2,3,4,5])
rfm['M']=pd.qcut(rfm['Total'],5,labels=[1,2,3,4,5])

#계산
rfm['RFM_score']=rfm['R'].astype(str)+rfm['F'].astype(str)+rfm['M'].astype(str)

In [21]:
rfm

,Seller_id,Recency,Frequency,Total,R,F,M,RFM_score
0,SELLER_0000,335,3,246.60,1,2,2,122
1,SELLER_0001,54,40,12823.32,3,5,5,355
2,SELLER_0002,259,1,199.92,1,1,1,111
3,SELLER_0003,83,1,73.89,3,1,1,311
4,SELLER_0004,5,35,4765.66,5,4,4,544
...,...,...,...,...,...,...,...,...
2756,SELLER_2758,17,1,93.85,4,1,1,411
2757,SELLER_2759,210,7,851.80,2,3,3,233
2758,SELLER_2760,9,4,505.58,5,2,2,522
2759,SELLER_2761,483,10,442.99,1,3,2,132


In [22]:
rfm.to_csv('RFM.csv',index=False)

In [25]:
#R
rfm.groupby('R').count()['Seller_id']

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_32113/67058907.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rfm.groupby('R').count()['Seller_id']


R
5    586
4    524
3    548
2    554
1    549
Name: Seller_id, dtype: int64

In [26]:
#F
rfm.groupby('F').count()['Seller_id']

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_32113/2875140164.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rfm.groupby('F').count()['Seller_id']


F
1    553
2    552
3    552
4    552
5    552
Name: Seller_id, dtype: int64

In [27]:
#
rfm.groupby(['M']).agg({'Seller_id':'count','Total':'sum'})

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_32113/1801244560.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rfm.groupby(['M']).agg({'Seller_id':'count','Total':'sum'})


,Seller_id,Total
M,,
1,553,58550.25
2,552,202193.00
3,552,577578.32
4,552,1736313.56
5,552,11461067.45


## RFM 시각화

R-F 행렬을 만들자, RFM 동시에 시각화하면 3차원이라서 가독성이 떨어진다. 

In [28]:
rf_matrix=rfm.groupby(['R','F']).agg({'Seller_id':'count','Total':'sum'}).reset_index()
rf_matrix['avg_sales']=rf_matrix['Total']/rf_matrix['Seller_id']
rf_matrix

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_32113/4009201608.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rf_matrix=rfm.groupby(['R','F']).agg({'Seller_id':'count','Total':'sum'}).reset_index()


,R,F,Seller_id,Total,avg_sales
0,5,1,34,8503.56,250.104706
1,5,2,72,41567.92,577.332222
2,5,3,96,134545.53,1401.515938
3,5,4,123,402226.84,3270.136911
4,5,5,261,6493358.58,24878.768506
5,4,1,55,9151.78,166.396000
6,4,2,92,48628.71,528.572935
7,4,3,90,166800.10,1853.334444
8,4,4,145,657089.79,4531.653724
9,4,5,142,2268339.78,15974.223803


In [29]:
rf_pivot=rf_matrix.pivot(index='F',columns='R',values='Seller_id').fillna(0).sort_index(ascending=False)
rf_pivot.style.background_gradient(cmap='Greens',axis=None,low=0.2)

R,5,4,3,2,1
F,,,,,
5,261,142,72,61,16
4,123,145,128,100,56
3,96,90,120,135,111
2,72,92,106,133,149
1,34,55,122,125,217


R-F 등급별로 속하는 사용자 수. 3-3이 가장 많다. 

In [30]:
#rf 등급별 평균 구매 금액
rf_pivot=rf_matrix.pivot(index='F',columns='R',values='avg_sales').fillna(0).sort_index(ascending=False)
rf_pivot.style.background_gradient(cmap='Greens',axis=None,low=0.2)

R,5,4,3,2,1
F,,,,,
5,24878.768506,15974.223803,14135.235000,11611.025574,6095.458125
4,3270.136911,4531.653724,3952.041328,3317.460500,3530.142321
3,1401.515938,1853.334444,1671.608417,1402.304519,1577.052162
2,577.332222,528.572935,688.866887,649.703759,666.339463
1,250.104706,166.396000,216.390656,239.604320,307.076912


- R=5,F=5 : 충성고객, 이탈하지 않도록 관리하자
- R=5,F=4 : 지출 금액이 가장 큼, 최근에 구매했고, 구매횟수도 꽤 많은 고객. 충성고객으로 만들기 위해 노력해야함
- R=5,F=3 : 지출금액이 그다음으로 큼. 더 자주 구매할 수 있도록 고객 맞춤 마케팅 적극적으로 하자!!!
- R=2,F=1 & R=1,F=1: 오래전에 구매했고 거의 몇번 구매 안한 고객. 근데 지출이 크다. 왜지. 아마 신규고객유치를 위한 특별 이벤트나 프로모션을 진행했을 때 구매했던 고객들인 듯. 다시 재방문할 수 있도록 재방문 시 파격적인 이벤트를 열거나 쿠폰을 뿌리자!!